In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import time

In [2]:
data_dir1=r'/content/drive/MyDrive/train/Concrete Data'
data_dir2=r'/content/drive/MyDrive/validation'

In [3]:
import pathlib
batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir1,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir2,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2000 files belonging to 2 classes.
Using 1600 files for training.
Found 384 files belonging to 2 classes.
Using 76 files for validation.


In [4]:
class_names = train_ds.class_names
print(class_names)

['Negative', 'Positive']


In [5]:
UTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=UTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=UTOTUNE)

In [6]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.


In [7]:
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from time import time
import numpy as np

In [8]:
def squeezenet(input_shape, n_classes):
  
  def fire(m, fs, fe):
    s = Conv2D(fs, 1, activation='relu')(m)
    e1 = Conv2D(fe, 1, activation='relu')(s)
    e3 = Conv2D(fe, 3, padding='same', activation='relu')(s)
    output = Concatenate()([e1, e3])
    return output
  
  
  input = Input(input_shape)
  
  m = Conv2D(96, 7, strides=2, padding='same', activation='relu')(input)
  m = MaxPool2D(3, strides=2, padding='same')(m)
  
  m = fire(m, 16, 64)
  m = fire(m, 16, 64)
  m = fire(m, 32, 128)
  m = MaxPool2D(3, strides=2, padding='same')(m)
  
  m = fire(m, 32, 128)
  m = fire(m, 48, 192)
  m = fire(m, 48, 192)
  m = fire(m, 64, 256)
  m = MaxPool2D(3, strides=2, padding='same')(m)
  
  m = fire(m, 64, 256)
  m = Conv2D(n_classes, 1)(m)
  m = GlobalAvgPool2D()(m)
  
  output = Activation('softmax')(m)
  
  model = Model(input, output)
  return model

In [8]:

def alexnet(input_shape, n_classes):
  input = Input(input_shape)
  
  # actually batch normalization didn't exist back then
  # they used LRN (Local Response Normalization) for regularization
  x = Conv2D(96, 11, strides=4, padding='same', activation='relu')(input)
  x = BatchNormalization()(x)
  x = MaxPool2D(3, strides=2)(x)
  
  x = Conv2D(256, 5, padding='same', activation='relu')(x)
  x = BatchNormalization()(x)
  x = MaxPool2D(3, strides=2)(x)
  
  x = Conv2D(384, 3, strides=1, padding='same', activation='relu')(x)
  
  x = Conv2D(384, 3, strides=1, padding='same', activation='relu')(x)
  
  x = Conv2D(256, 3, strides=1, padding='same', activation='relu')(x)
  x = BatchNormalization()(x)
  x = MaxPool2D(3, strides=2)(x)
  
  x = Flatten()(x)
  x = Dense(4096, activation='relu')(x)
  x = Dense(4096, activation='relu')(x)
  
  output = Dense(n_classes, activation='softmax')(x)
  
  model1 = Model(input, output)
  return model1

In [9]:
'''
input_shape=(img_height, img_width, 3)
n_classes =2
K.clear_session()
model = squeezenet(input_shape, n_classes)
model.summary()
'''


input_shape=(img_height, img_width, 3)
n_classes =2
K.clear_session()
model1 = alexnet(input_shape, n_classes)
model1.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 180, 180, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 45, 45, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 45, 45, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 22, 22, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 256)       0     

In [11]:
model1.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [13]:
epochs=5
history = model1.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/5
50/50 [==============================] - 201s 4s/step - loss: 12.7518 - accuracy: 0.8252 - val_loss: 20.4373 - val_accuracy: 0.5263
Epoch 2/5
50/50 [==============================] - 162s 3s/step - loss: 4.0521 - accuracy: 0.8985 - val_loss: 2.1697 - val_accuracy: 0.5526
Epoch 3/5
50/50 [==============================] - 161s 3s/step - loss: 0.0515 - accuracy: 0.9819 - val_loss: 0.4826 - val_accuracy: 0.9079
Epoch 4/5
50/50 [==============================] - 161s 3s/step - loss: 0.0277 - accuracy: 0.9909 - val_loss: 0.3654 - val_accuracy: 0.9211
Epoch 5/5
50/50 [==============================] - 163s 3s/step - loss: 0.0145 - accuracy: 0.9931 - val_loss: 0.1131 - val_accuracy: 0.9474
